In [14]:
# Importing necessary libraries
import enum
import itertools
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Optional

# Defining musical intervals as an enumeration
class Interval(enum.Enum):
    UNISON = 0
    MINOR_SECOND = 1
    MAJOR_SECOND = 2
    MINOR_THIRD = 3
    MAJOR_THIRD = 4
    PERFECT_FOURTH = 5
    TRITONE = 6
    PERFECT_FIFTH = 7
    MINOR_SIXTH = 8
    MAJOR_SIXTH = 9
    MINOR_SEVENTH = 10
    MAJOR_SEVENTH = 11

# Defining chord qualities as an enumeration
class ChordQuality(enum.Enum):
    MAJOR = (Interval.UNISON, Interval.MAJOR_THIRD, Interval.PERFECT_FIFTH)
    MINOR = (Interval.UNISON, Interval.MINOR_THIRD, Interval.PERFECT_FIFTH)
    DIMINISHED = (Interval.UNISON, Interval.MINOR_THIRD, Interval.TRITONE)
    AUGMENTED = (Interval.UNISON, Interval.MAJOR_THIRD, Interval.MINOR_SIXTH)

# Defining a Note class to represent musical notes
class Note:
    def __init__(self, name: str, octave: int):
        self.name = name
        self.octave = octave

    def __str__(self):
        return f'{self.name}{self.octave}'

    def __lt__(self, other):
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        if self.octave != other.octave:
            return self.octave < other.octave
        return notes.index(self.name) < notes.index(other.name)

class Fretboard:
    def __init__(self, tuning):
        self.tuning = tuning
        self.create_fretboard()

    def create_fretboard(self):
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        num_strings = len(self.tuning)
        num_frets = 12  # You can adjust this if needed
        self.fretboard = np.empty((num_strings, num_frets), dtype=object)
        for i, string in enumerate(self.tuning):
            start_index = notes.index(string.name)
            for fret in range(num_frets):
                note_index = (start_index + fret) % 12
                octave = string.octave + (start_index + fret) // 12
                self.fretboard[i][fret] = Note(notes[note_index], octave)

    def print_fretboard(self):
        print("Fretboard:")
        print(" ", end="")
        for fret in range(12):
            print(f"{fret:2d} ", end="")
        print()
        for string, tuning_note in enumerate(self.tuning):
            print(f"{tuning_note}: ", end="")
            for fret in range(12):
                note = self.fretboard[string][fret]
                print(f"{note.name:2s} ", end="")
            print()

    def get_pitch(self, string: int, fret: int) -> Note:
        return self.fretboard[string][fret]


@dataclass
class ChordDiagram:
    frets: List[int]
    fingers: List[int]
    mutes: List[bool]
    root: str
    quality: ChordQuality
    fretboard: Fretboard

    def __post_init__(self):
        self.notes = self.get_notes_with_pitches()
        self.root_note = self.find_root_note()
        self.bass_note = self.notes[0] if self.notes else None
        self.inversion = self.find_inversion()
        self.is_partial = len(self.notes) < 3  # Assuming a triad

    def get_notes_with_pitches(self):
        return [self.fretboard.get_pitch(string, fret) for string, fret in enumerate(self.frets) if fret >= 0]

    def find_root_note(self):
        for note in self.notes:
            if note.name == self.root:
                return note
        return None

    def get_unique_id(self):
        return tuple(self.frets)

    def get_position(self):
        played_frets = [f for f in self.frets if f >= 0]
        if not played_frets:
            return "Open"
        min_fret = min(played_frets)
        max_fret = max(played_frets)
        if min_fret == 0:
            return "Open"
        elif min_fret == max_fret and self.fingers.count(1) >= 3:
            return f"Barre at fret {min_fret}"
        else:
            return f"Fret {min_fret}"

    def get_notes_with_strings(self):
        return [f"{self.fretboard.tuning[i].name}{self.fretboard.tuning[i].octave}:{note}" for i, note in enumerate(self.notes)]

    def find_inversion(self):
        if not self.root_note or not self.notes:
            return 0
        return self.notes.index(self.root_note)

    def get_chord_name(self):
        inversion_names = ['', ' (first inversion)', ' (second inversion)']
        return f"{self.root} {self.quality.name}{inversion_names[self.inversion]}"

    def __lt__(self, other):
        return self.bass_note < other.bass_note if self.bass_note and other.bass_note else False


class GuitarChordGenerator:
    def __init__(self, tuning=['G3', 'D4', 'A4', 'E5']):  # GDAE tuning for tenor banjo
        self.tuning = [Note(name=note[:-1], octave=int(note[-1])) for note in tuning]
        self.fretboard = Fretboard(self.tuning)

    def generate_all_triads(self, root: str):
        triads = []
        qualities = [ChordQuality.MAJOR, ChordQuality.MINOR, ChordQuality.DIMINISHED, ChordQuality.AUGMENTED]
        for quality in qualities:
            triads.extend(self.generate_chord(root, quality))
        return triads

    def generate_chord(self, root: str, quality: ChordQuality, full_chord=True, partial_chord=True, inversions=True, open_strings=True, max_fret_span=4) -> List[ChordDiagram]:
        chord_notes = self._get_chord_notes(root, quality)
        shapes = self._build_shape(chord_notes, full_chord, partial_chord, inversions, open_strings, max_fret_span)
        unique_diagrams = {}
        for shape in shapes:
            diagram = self._create_chord_diagram(shape, root, quality)
            unique_id = diagram.get_unique_id()
            if unique_id not in unique_diagrams:
                unique_diagrams[unique_id] = diagram
        return list(unique_diagrams.values())

    def _create_chord_diagram(self, shape, root: str, quality):
        frets = [-1] * 4
        fingers = [0] * 4
        mutes = [True] * 4
        for string, fret in shape:
            frets[string] = fret
            fingers[string] = 1 if fret > 0 else 0
            mutes[string] = False
        return ChordDiagram(frets, fingers, mutes, root, quality, self.fretboard)

    def _get_chord_notes(self, root: str, quality: ChordQuality) -> List[str]:
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        root_index = notes.index(root)
        return [notes[(root_index + interval.value) % 12] for interval in quality.value]

    def _build_shape(self, chord_notes: List[str], full_chord=True, partial_chord=True, inversions=True, open_strings=True, max_fret_span=4) -> List[List[Tuple[int, int]]]:
        shapes = []
        if full_chord:
            full_shapes = self._find_full_chord(chord_notes, open_strings, max_fret_span)
            shapes.extend(full_shapes)
        if partial_chord:
            partial_shapes = self._find_partial_chord(chord_notes, open_strings, max_fret_span)
            shapes.extend(partial_shapes)
        if inversions:
            inversion_shapes = self._find_inversions(chord_notes, open_strings, max_fret_span)
            shapes.extend(inversion_shapes)
        return shapes

    def _find_full_chord(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for combo in itertools.combinations(range(4), len(chord_notes)):
            shape = []
            for i, string in enumerate(combo):
                if open_strings and self.fretboard.get_pitch(string, 0).name == chord_notes[i]:
                    shape.append((string, 0))
                else:
                    for fret in range(12):  # Search all frets up to 12
                        if self.fretboard.get_pitch(string, fret).name == chord_notes[i]:
                            shape.append((string, fret))
                            break
            if len(shape) == len(chord_notes) and self._is_valid_shape(shape, max_fret_span):
                shapes.append(shape)
        return shapes

    def _find_partial_chord(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for r in range(1, len(chord_notes)):  # Start from 1 to include single-note "chords"
            for subset in itertools.combinations(chord_notes, r):
                shapes.extend(self._find_full_chord(subset, open_strings, max_fret_span))
        return shapes

    def _find_inversions(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for inversion in itertools.permutations(chord_notes):
            shapes.extend(self._find_full_chord(inversion, open_strings, max_fret_span))
        return shapes

    def _is_valid_shape(self, shape: List[Tuple[int, int]], max_fret_span: int) -> bool:
        frets = [fret for _, fret in shape if fret > 0]
        return not frets or max(frets) - min(frets) <= max_fret_span


In [16]:
# Main execution block
if __name__ == "__main__":
    # Create a GuitarChordGenerator instance
    generator = GuitarChordGenerator()
    
    # Set the root note and chord qualities
    root = 'C'
    qualities = [ChordQuality.MAJOR, ChordQuality.MINOR, ChordQuality.DIMINISHED, ChordQuality.AUGMENTED]
    
    # Generate all triads
    all_triads = []
    for quality in qualities:
        all_triads.extend(generator.generate_chord(root, quality, full_chord=True, partial_chord=False, inversions=True, open_strings=True, max_fret_span=4))
    
    # Sort the triads by bass note
    sorted_triads = sorted(all_triads)
    
    # Print the results
    print(f"\n{root} Triads (sorted by bass note):")
    for i, diagram in enumerate(sorted_triads, 1):
        if diagram.quality.name == 'MAJOR' and diagram.inversion == 0:
            print(f"Diagram {i}:")
            print(f"Chord: {diagram.get_chord_name()}")
            print(f"Frets: {diagram.frets}")
            print(f"Fingers: {diagram.fingers}")
            print(f"Mutes: {diagram.mutes}")
            print(f"Root Note: {diagram.root_note}")
            print(f"Bass Note: {diagram.bass_note}")
            print(f"Inversion: {diagram.inversion}")
            print(f"Is Partial: {diagram.is_partial}")
            print(f"Notes: {', '.join(str(note) for note in diagram.notes)}")
            print(f"Position: {diagram.get_position()}")
            print(f"Notes on strings: {', '.join(diagram.get_notes_with_strings())}")
            print()

    # Print the fretboard
    generator.fretboard.print_fretboard()



C Triads (sorted by bass note):
Diagram 12:
Chord: C MAJOR
Frets: [5, 2, -1, 3]
Fingers: [1, 1, 0, 1]
Mutes: [False, False, True, False]
Root Note: C4
Bass Note: C4
Inversion: 0
Is Partial: False
Notes: C4, E4, G5
Position: Fret 2
Notes on strings: G3:C4, D4:E4, A4:G5

Diagram 13:
Chord: C MAJOR
Frets: [5, -1, 7, 3]
Fingers: [1, 0, 1, 1]
Mutes: [False, True, False, False]
Root Note: C4
Bass Note: C4
Inversion: 0
Is Partial: False
Notes: C4, E5, G5
Position: Fret 3
Notes on strings: G3:C4, D4:E5, A4:G5

Diagram 14:
Chord: C MAJOR
Frets: [5, 5, 7, -1]
Fingers: [1, 1, 1, 0]
Mutes: [False, False, False, True]
Root Note: C4
Bass Note: C4
Inversion: 0
Is Partial: False
Notes: C4, G4, E5
Position: Fret 5
Notes on strings: G3:C4, D4:G4, A4:E5

Diagram 15:
Chord: C MAJOR
Frets: [5, 5, -1, 0]
Fingers: [1, 1, 0, 0]
Mutes: [False, False, True, False]
Root Note: C4
Bass Note: C4
Inversion: 0
Is Partial: False
Notes: C4, G4, E5
Position: Open
Notes on strings: G3:C4, D4:G4, A4:E5

Diagram 49:
Chord